In [2]:
import sys
import glob
import cv2
# 이미지 파일을 모두 img_files 리스트에 추가
img_files = glob.glob('./data/images/*.jpg')
if not img_files:
    print("There are no jpg files in 'images' folder")
    sys.exit()
# 전체 화면으로 'image' 창 생성
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('image', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# 무한 루프
cnt = len(img_files)
idx = 0
while True:
    img = cv2.imread(img_files[idx])
    if img is None:
        print('Image load failed!')
        break
    cv2.imshow('image', img)
    if cv2.waitKey(2000) >= 0:
        break
    idx += 1
    if idx >= cnt:
        idx = 0
cv2.destroyAllWindows()

In [ ]:
import sys
import numpy as np
import cv2
# 두 개의 동영상을 열어서 cap1, cap2로 지정
cap1 = cv2.VideoCapture('./data/video/video1.mp4')
cap2 = cv2.VideoCapture('./data/video/video2.mp4')
if not cap1.isOpened() or not cap2.isOpened():
    print('video open failed!')
    sys.exit()
# 두 동영상의 크기, FPS는 같다고 가정함
frame_cnt1 = round(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
frame_cnt2 = round(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap1.get(cv2.CAP_PROP_FPS)
effect_frames = int(fps * 2)
print('frame_cnt1:', frame_cnt1)
print('frame_cnt2:', frame_cnt2)
print('FPS:', fps)
delay = int(1000 / fps)
w = round(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# 출력 동영상 객체 생성
out = cv2.VideoWriter('output.avi', fourcc, fps, (w, h))
# 1번 동영상 복사
for i in range(frame_cnt1 - effect_frames):
    ret1, frame1 = cap1.read()
    if not ret1:
        print('frame read error!')
        sys.exit()
    out.write(frame1)
    print('.', end='')
    cv2.imshow('output', frame1)
    cv2.waitKey(delay)
# 1번 동영상 뒷부분과 2번 동영상 앞부분을 합성
for i in range(effect_frames):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if not ret1 or not ret2:
        print('frame read error!')
        sys.exit()
    dx = int(w / effect_frames) * i
    frame = np.zeros((h, w, 3), dtype=np.uint8)
    frame[:, 0:dx, :] = frame2[:, 0:dx, :]
    frame[:, dx:w, :] = frame1[:, dx:w, :]
    #alpha = i / effect_frames
    #frame = cv2.addWeighted(frame1, 1 - alpha, frame2, alpha, 0)
    out.write(frame)
    print('.', end='')
    cv2.imshow('output', frame)
    cv2.waitKey(delay)
# 2번 동영상을 복사
for i in range(effect_frames, frame_cnt2):
    ret2, frame2 = cap2.read()
    if not ret2:
        print('frame read error!')
        sys.exit()
    out.write(frame2)
    print('.', end='')
    cv2.imshow('output', frame2)
    cv2.waitKey(delay)
print('\noutput.avi file is successfully generated!')
cap1.release()
cap2.release()
out.release()
cv2.destroyAllWindows()